In [1]:
from selenium import webdriver
import time
import pandas as pd
import pymongo as mg

In [2]:
#open chrome browser
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1560,840")
browser = webdriver.Chrome(executable_path='../../../chromedriver.exe', options=options) #webdriver_selenium과 web을 연결해주기위함. 

In [3]:
# url in address window
browser.get('https://www.coupang.com/np/search?component=&q=%EB%88%88+%EC%98%81%EC%96%91%EC%A0%9C&channel=user')
browser.implicitly_wait(10)

In [4]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_nutrients']
collection = database['eyes_review']